In [ ]:
# Instructions
# Do not forget to download the bills as CSV files in the download folder "invoices"
# https://github.com/senadjukic/confluent-cloud-invoices-viz

In [ ]:
# Check if required packages are installed
# !pip3 install pandas matplotlib numpy
# !pip3 install -r requirements.txt

# In case you want to save your package list
# !pip3 list --format=freeze > requirements.txt

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display
import numpy as np
import os
import glob
from datetime import date

# Use glob to get all paths of the csv files from the download folder
path = os.getcwd()
download_folder = "invoices"
csv_files = glob.glob(os.path.join(path, download_folder, "*.csv"))

# Check if no CSV's were found
if not csv_files:
   raise UserWarning('No downloaded CSV found in the folder:',download_folder)
else:
   print('Downloaded CSV found in folder:',download_folder)

# Placeholder for all months
li = []
    
# Loop over the list of CSV files
for f in csv_files:
      
    # Read the csv file
    df = pd.read_csv(f) 
      
    # Print the location and filename
    print('Processed:', f)
    month_year = '_'.join(f.split('_')[-2:]).removesuffix('.csv')
      
    # Group spend by product per monthly bill
    spend_by_month = df.groupby("Product")
    
    # Round monthly product spend to 2 decimals
    spend_by_month = spend_by_month["Total"].sum(numeric_only=True).round(2)
    
    # Rename the column from CSV filename to month_year
    spend_by_month = spend_by_month.rename(month_year)
    li.append(spend_by_month)

# All months alphabetically sorted
spend_by_month_concat = pd.concat(li, axis=1)

In [ ]:
# Sort all columns by years and months
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

# Add x Axis values in the right timewise order to the list, such as December_2022, January_2023, February_2023, and similar
years = ['2021','2022','2023']

months_with_years = []

for year in years:
    for month in months:
        months_with_years.append(month + "_" + year)

# Remove all months that have only NaN values
spend_by_month_concat = spend_by_month_concat.reindex(columns=months_with_years).dropna(axis=1, how='all')

# Add Monthly Totals
spend_by_month_concat.loc['Monthly Total'] = spend_by_month_concat.sum(numeric_only=True, axis=0)
display(spend_by_month_concat)

In [ ]:
# Billing per service: ['connect', 'kafka', 'ksql', 'event-log', 'stream-governance']
billable_services = spend_by_month_concat.index.values

# Plot the spends by service
for service in billable_services: 
    fig, ax = plt.subplots()
    x_axis = spend_by_month_concat.columns
    y_axis = spend_by_month_concat.T[service]
    ax.bar(x_axis, y_axis, label=x_axis)
    ax.set_ylabel('$/month')
    ax.set_title(service +' Costs')
    plt.rcParams['figure.figsize'] = [25, 5]
    plt.show()

In [ ]:
# Add Service Totals
spend_by_month_concat.loc[:,'Service_Total'] = spend_by_month_concat.sum(numeric_only=True, axis=1)

# Get sum of Service_Total aka total spend on all services
total_costs = spend_by_month_concat['Service_Total'][-1]

service_total_percentage = []

# Ratio service spend (Service_Total) against total spend on all services (total_costs)
for index in range(len(billable_services)): 
    service_costs_percentage_from_total_costs = spend_by_month_concat['Service_Total'][index] / total_costs * 100
    #print(billable_services[index], round(service_costs_percentage_from_total_costs,2)," %")
    service_total_percentage.append(str(round(service_costs_percentage_from_total_costs,2)) + " %")

spend_by_month_concat['Service_Total_Percentage'] = service_total_percentage

# Display table
display(spend_by_month_concat)